In [14]:
import os

import pandas as pd

## toplines & visuals
# from pandas_profiling import ProfileReport
import sweetviz as sv

import numpy as np

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_colwidth', None)

## set notebook to 100% width of page
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Data

In [15]:
surveys = pd.read_csv('./datasets/dpp/2016-survey-data-kaggle/VOTER_Survey_December16_Release1.csv')
surveys.columns = surveys.columns.str.lower()

q1 = pd.read_csv('../job-apply/interviews/gsg/data/question1_data.csv')

questions = pd.read_csv('../job-apply/interviews/gsg/data/question4_files/questions.csv')
responses = pd.read_csv('../job-apply/interviews/gsg/data/question4_files/responses.csv')
weight_schemes = pd.read_csv('../job-apply/interviews/gsg/data/question4_files/weight_schemes.csv')
respondent_data = pd.read_csv('../job-apply/interviews/gsg/data/question4_files/respondent_data.csv')
respondent_weight = pd.read_csv('../job-apply/interviews/gsg/data/question4_files/respondent_weights.csv')

/home/patruq/anaconda3/envs/dpp/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (423,564) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Toplines

In [7]:
## QUESTION - RESPONSE - FREQUENCY - PERCENTAGE

In [8]:
q_18058 = questions[questions['project_id'] == 18058].sort_values('question_id')
q_18390 = questions[questions['project_id'] == 18390].sort_values('question_id')

r_18058 = responses[responses['project_id'] == 18058].sort_values('question_id')
r_18390 = responses[responses['project_id'] == 18390].sort_values('question_id')

rd_18058 = respondent_data[respondent_data['project_id'] == 18058].sort_values('question_id')
rd_18390 = respondent_data[respondent_data['project_id'] == 18390].sort_values('question_id')

rw_18058 = respondent_weight[respondent_weight['project_id'] == 18058]
rw_18390 = respondent_weight[respondent_weight['project_id'] == 18390]

ws_18058 = weight_schemes[weight_schemes['project_id'] == 18058]
ws_18390 = weight_schemes[weight_schemes['project_id'] == 18390]

In [9]:
## combine questions & response on 'responses'
output_18058 = r_18058.merge(q_18058, how='left', on='question_id')
## remove unneeded cols
output_18058 = output_18058[['question_id', 'response', 
                             'response_index', 'text_y','text_x']]
output_18058.rename(columns={
    'text_y': 'question_text',
    'text_x': 'response_text'
}, inplace=True)

## order the cols by question and respnse
output_18058.sort_values(['question_id', 'response'], inplace=True)

## corr resp_weights to weight scheme marked 'final' in weight_schemes
cond = (ws_18058['name'] == 'Final')
w_id = ws_18058[cond]['weight_scheme_id'].values[0]

rw_18058 = rw_18058[rw_18058['weight_scheme_id'] == w_id]

In [10]:
## weights are tied to individual respondent id's
## For more info/background on weighted tables in survey data, ref'd from source:
    ## https://www.datarevelations.com/working-with-weighted-survey-data/
## perc is the % of overall choice in category/question

## combine the weights corr to respondent, response, and question.
m_weights_18058 = rd_18058.merge(rw_18058, how='left', 
                                 on='respondent_id').sort_values('respondent_id').groupby(['question_id', 
                                                                                           'response'], 
                                                                                           as_index=False).sum()
## merge with questions/respondents data
output_18058 = pd.merge(output_18058, m_weights_18058, how='left', 
                        left_on=['question_id', 'response'],
                        right_on=['question_id', 'response'])

## clean cols, round floats down
output_18058.rename(columns={'weight': 'freq_weight'}, inplace=True)
output_18058['freq_weight'].fillna(0, inplace=True)
output_18058['freq_weight'] = output_18058['freq_weight'].apply(lambda x: np.round(x, 1))

## the meat and potatoes:
cumsum = output_18058.groupby(['question_id']).sum().groupby(level=0).cumsum().reset_index()
output_18058 = output_18058.merge(cumsum, how='left', on='question_id')

output_18058['percent_weighted'] = output_18058['freq_weight_x'] / output_18058['freq_weight_y']

output_18058 = output_18058[['question_id', 'question_text', 'response_text', 
                             'freq_weight_x', 'percent_weighted']].rename(columns={'freq_weight_x': 'freq_weighted'})
output_18058

,question_id,question_text,response_text,freq_weighted,percent_weighted
0,A19,Shares your values,Very well,30.8,0.152929
1,A19,Shares your values,Somewhat well,71.4,0.354518
2,A19,Shares your values,Not very well,23.1,0.114697
3,A19,Shares your values,Not well at all,43.4,0.215492
4,A19,Shares your values,(Don't know/Refused),32.7,0.162363
...,...,...,...,...,...
248,V1,"If the election for US Congress was today, for...",John Crichton,125.7,0.311833
249,V1,"If the election for US Congress was today, for...",Lean John Crichton,4.4,0.010915
250,V1,"If the election for US Congress was today, for...",(Undecided/Refused),58.3,0.144629
251,VER,VERSION,A,201.5,0.500000


# Crosstabs, Pivot Tables

In [16]:
print(surveys.shape)
surveys.head()

(8000, 668)


,case_identifier,weight,party_agendas_rand_2016,pp_primary16_2016,pp_demprim16_2016,pp_repprim16_2016,inputstate_2016,izip_2016,votereg2_2016,votereg_f_2016,regzip_2016,turnout16_2016,vote_usual_2016,votemeth16_2016,presvote16post_2016,presvote16post_t_2016,vote2016_cand2_2016,vote_for_against_2016,clinton_rubio_2016,clinton_cruz_2016,sanders_trump_2016,sanders_rubio_2016,wished_voted_2016,vote_regrets_2016,second_chance_2016,second_chance_t_2016,accurately_counted_2016,accurately_counted2_2016,obamaapp_2016,fav_trump_2016,fav_cruz_2016,fav_ryan_2016,fav_romn_2016,fav_obama_2016,fav_hrc_2016,fav_sanders_2016,fav_rubio_2016,party_agendas_d1_2016,party_agendas_d2_2016,party_agendas_d3_2016,party_agendas_d4_2016,party_agendas_d5_2016,party_agendas_d6_2016,party_agendas_d7_2016,party_agendas_d8_2016,party_agendas_d9_2016,party_agendas_d10_2016,party_agendas_d11_2016,party_agendas_d12_2016,party_agendas_r1_2016,party_agendas_r2_2016,party_agendas_r3_2016,party_agendas_r4_2016,party_agendas_r5_2016,party_agendas_r6_2016,party_agendas_r7_2016,party_agendas_r8_2016,party_agendas_r9_2016,party_agendas_r10_2016,party_agendas_r11_2016,party_agendas_r12_2016,rigged_system_1_2016,rigged_system_2_2016,rigged_system_3_2016,rigged_system_4_2016,rigged_system_5_2016,rigged_system_6_2016,track_2016,persfinretro_2016,econtrend_2016,americatrend_2016,futuretrend_2016,wealth_2016,values_culture_2016,us_respect_2016,trustgovt_2016,trust_people_2016,helpful_people_2016,fair_people_2016,ft_black_2016,ft_white_2016,ft_hisp_2016,ft_asian_2016,ft_muslim_2016,ft_jew_2016,ft_christ_2016,ft_fem_2016,ft_immig_2016,ft_blm_2016,ft_wallst_2016,ft_gays_2016,ft_unions_2016,ft_police_2016,ft_altright_2016,imiss_a_2016,imiss_b_2016,imiss_c_2016,imiss_d_2016,imiss_e_2016,imiss_f_2016,imiss_g_2016,imiss_h_2016,imiss_i_2016,imiss_j_2016,imiss_k_2016,imiss_l_2016,imiss_m_2016,imiss_n_2016,imiss_o_2016,imiss_p_2016,imiss_q_2016,imiss_r_2016,imiss_s_2016,imiss_t_2016,imiss_u_2016,imiss_x_2016,imiss_y_2016,immi_contribution_2016,immi_naturalize_2016,immi_makedifficult_2016,immi_muslim_2016,abortview3_2016,gaymar_2016,view_transgender_2016,deathpen_2016,deathpenfreq_2016,police_threat_2016,conviction_accuracy_2016,univhealthcov_2016,healthreformbill_2016,envwarm_2016,envpoll2_2016,affirmact_gen_2016,taxdoug_2016,govt_reg_2016,gvmt_involment_2016,tradepolicy_2016,free_trade_1_2016,free_trade_2_2016,free_trade_3_2016,free_trade_4_2016,free_trade_5_2016,amcitizen_2016,amshamed_2016,belikeus_2016,prouddem_2016,proudhis_2016,proudgrp_2016,ambornin_2016,amcit_2016,amlived_2016,amenglish_2016,amchrstn_2016,amgovt_2016,amwhite_2016,amdiverse_2016,sexism1_2016,sexism2_2016,sexism3_2016,sexism4_2016,sexism5_2016,sexism6_2016,gender_equality_2016,race_deservemore_2016,race_overcome_2016,race_tryharder_2016,race_slave_2016,policies_favor_2016,reverse_discrimination_2016,inc_opp_blacks_2016,race_majority_1_2016,race_majority_2_2016,race_majority_3_2016,race_majority_4_2016,social_conformity_1_2016,social_conformity_2_2016,social_conformity_3_2016,social_conformity_4_2016,civic_participation_2016,political_correctness_2016,race_2016,race_other_2016,race_fate_2016,race_importance_2016,labunmemb_2016,milstat_1_2016,milstat_2_2016,milstat_3_2016,milstat_4_2016,milstat_5_2016,gunown_2016,smoke100_2016,smokenow_2016,healthdk_0_2016,healthcov2_2016,healthcov_t_2016,alcohol_2016,painkillers_2016,drugs_2016,victim_12mo_2016,educ_2016,marstat_2016,child18_2016,employ_2016,employ_t_2016,faminc_2016,pid3_2016,pid3_t_2016,pid7_2016,ideo5_2016,newsint_2016,pew_bornagain_2016,pew_religimp_2016,pew_churatd_2016,pew_prayer_2016,religpew_2016,religpew_t_2016,votemeth16_rnd_2016,presvote16post_rnd_2016,vote2016_cand2_rnd_2016,clinton_rubio_rnd_2016,clinton_cruz_rnd_2016,sanders_trump_rnd_2016,sanders_rubio_rnd_2016,second_chance_rnd_2016,obamaapp_rnd_2016,fav_grid_row_rnd_2016,fav_grid_col_rnd_2016,party_agendas_d_grid_row_rnd_201,party_agendas_r_grid_row_rnd_201,rigged_system_grid_row_rnd_2016,track_rn

In [17]:
cols = [col for col in surveys.columns if 'fav' in col and '_2016' in col] 
fav = surveys[cols].copy().drop(columns={'policies_favor_2016', 'fav_grid_row_rnd_2016', 'fav_grid_col_rnd_2016'})
fav.fillna('.', inplace=True)

## agg count the freq of values
fav = fav.apply(pd.value_counts)

fav.columns = ['Trump', 'Cruz', 'Ryan', 'Romney',
               'Obama', 'Clinton', 'Sanders', 'Rubio']

## swap the col and index of df
fav = fav.T

## convert to string with '%'
total = surveys.shape[0]
fav = fav.apply(lambda x: (np.round(x / total, 2)*100).astype(int).astype(str) + '%')

## reorder cols
fav = fav[['Very favorable', 'Somewhat favorable', 'Somewhat unfavorable', 'Very unfavorable', '.']]
fav

,Very favorable,Somewhat favorable,Somewhat unfavorable,Very unfavorable,.
Trump,26%,18%,8%,45%,1%
Cruz,11%,24%,18%,40%,1%
Ryan,9%,27%,24%,32%,1%
Romney,6%,28%,35%,24%,1%
Obama,34%,15%,9%,41%,1%
Clinton,22%,20%,9%,48%,1%
Sanders,27%,26%,16%,26%,1%
Rubio,12%,30%,24%,25%,1%


In [ ]:
## across all Q's
## age, gender, race, education, party ID, idealogy, 2016 vote choice, Dem vote choice, GOP vote choice
reg_col = 'votereg2_2016'
age_col = 'age'
gender_col = 'gender_baseline'
race_col = 'race_baseline'
edu_col = 'educ_baseline'
party_col = 'partyreg_baseline'
id_col = 'ideo5_2016'
vote_col = 'presvote16post_2016'
dem_col = 'pp_demprim16_2016'
gop_col = 'pp_repprim16_2016'

## create 'age' col 
tables = surveys.copy()

tables['age'] = (2016 - tables['birthyr_baseline'])

## CROSSTABS
    ## quantiles (buckets) for the ages
    ## NEED THIS FOR EITHER APPROACH
bins = [18, 30, 45, 64, np.inf]
labels = ['18-29', '30-44', '45-64', '65+']
tables['age_quant'] = pd.cut(tables['age'], bins=bins, labels=labels)

## LONG WAY
## group answers with age_quants
output = pd.DataFrame(tables.groupby('age_quant')[reg_col].value_counts()).unstack(level=[0])
## reset col names
output.columns = labels
## change col name
output.index.name = 'Voter Registration Status'
## sort index
output.sort_index(ascending=False, inplace=True)

## SHORT WAY
pd.crosstab(index=tables[reg_col], columns=tables['age_quant'])
    ## WITH %
pd.crosstab(index=tables[reg_col], columns=tables['age_quant']).apply(lambda x: (x/x.sum())*100, axis=1)

In [33]:
    ## BUCKETS/QUANTS FOR CATEGORICAL DATA
tables.groupby(gender_col)[reg_col].value_counts().unstack(level=[0])
    ## or-
pd.crosstab(index=tables[gender_col], columns=tables[reg_col]).T

gender_baseline,Female,Male
votereg2_2016,,
Don't know,13,8
No,45,25
Yes,4002,3907


# Matches btwn cols

## map to new col based on value in other cols in 1 df

In [ ]:
## ref = use output from cleaning

output['age'] = output['age'].astype(int)

date_year = output['startdate'].str[-4].astype(int)
cond1 = (abs((date_year - output['d101']) - output['age']) == 1) | (abs((date_year - output['d101']) - output['age']) == 0)
cond2 = (output['sex'].map(lambda x: 2 if 'F' in x else 1) == output['d100'])

## new binary column of matches or mismatches
output['voter_file_match'] = (cond2 | (np.where(~cond1, False, True)))
output

# EDA Kickstarters

In [34]:
report = sv.analyze(q1)
report.show_html()

                                             |                                                                …

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


# Cleaning

## split, strip values in col into new cols for each value

In [ ]:
## clean cols
q1.columns = q1.columns.str.lower()

## mask col
col = q1['vf_data']
## split the data in the col
split = col.apply(lambda x: x.split(';'))

## create dictionary of split values in previous col
dct = [dict(x.split('=', 1) for x in y) for y in split]

## merge dct into data
output = q1.join(pd.DataFrame(dct)).drop(columns='vf_data')
output.columns = output.columns.str.lower()

## order cols alphabetically

In [ ]:
col_order = ['origin']
new_order = col_order + (output.columns.drop(col_order).tolist())
output = output[new_order]

## BELOW
output = output.reindex(sorted(output.columns), axis=1)

#